In [6]:
import pandas as pd
import pathlib
import numpy as np
from numpy import log as ln
import scipy.stats as stats
from scipy import ndimage
import multiprocessing
import time

In [7]:
def walk(path):
    """Collects paths for files in path parameter

    Parameters
    ----------
    path: str or pathlib.Path() object
            Path to metadata folder containing IDR study directories

    Returns
    -------
    PosixPath object
    """
    for p in pathlib.Path(path).iterdir(): 
        if p.is_dir(): 
            yield from walk(p)
            continue
        yield p.resolve()


In [19]:
# Define stats
# Relative and absolute frequencies
def category_frequencies(attribute_elements):
    """Calculates absolute and relative frequencies for unique elements of an image attribute

    Parameters
    ----------
    attribute_elements: dict
        Unique elements of an image attribute as keys and instances of each as values

    Returns
    -------
    rel_freq_dict: dict
        Relative frequencies per unique image attribute element
    abs_freq-list: list
        Instance counts of unique image attribute elements
    """
    total_instances = sum(attribute_elements.values())
    rel_freq_dict = dict()
    abs_freq_list = list()
    for image_attribute in attribute_elements.keys():
        abs_freq_list.append(attribute_elements[image_attribute])
        rel_freq_dict[image_attribute] = attribute_elements[image_attribute] / total_instances
    return rel_freq_dict, abs_freq_list
    
# Shannon Index
def h_index(p):
    """Calculates the Shannon Index of a set of unique attribute instances.

    Parameters
    ----------
    p: dict
        Dictionary of relative frequencies for each unique element in an attribute column.

    Returns
    -------
    h: float
        Shannon Index value  
    results: list
        List of each -p_iln(p_i) value to use for Normalized Median Evenness statistic.
    """
    results = list()
    for entry in p.values():
        results.append(entry * ln(entry))
    
    results = np.array(results)
    h = -(sum(results))
    return h, results

# Pielou's Evenness
def pielou(h, s):
    """Calculates the ratio of the observed Shannon Index to the maximum possible Shannon Index value
    
    Parameters
    ----------
    h: float
        Observed Shannon Index calculated by h-index()
    s: int
        Observed richness within an image attribute (count of unique entries)

    Returns
    -------
    j: float
        Pielou's evenness (H_obs / H-max)

    """
    j = h / ln(s)
    return j

# Normalized Median Evenness
def norm_median_evenness(h_list):
    """Calculates Normalized Median Evenness (NME) of Shannon Index summation elements (-p*ln(p))
    
    Parameters
    ----------
    h_list: list
        Individual -p*ln(p) values of the Shannon Index summation

    Returns
    -------
    nme: float
        Ratio of median and max -p*ln(p) values
    """
    temp_list = list()
    for h_value in h_list:
        temp_list.append(-1.0 * h_value)
    temp_list = np.array(temp_list)
    nme = ndimage.median(temp_list) / temp_list.max()
    return nme

def gini_coef(absolute_frequencies_list):
    """Calculates the Gini coefficient of inequality across unique elements per image attribute

    Parameters
    ----------
    absolute_frequencies_list: list
        Counts of instances of each unique element of an image attribute

    Returns
    -------
    gc: float
        Measure of inequality in range [0, 1] where 0 is perfect equality and 1 is perfect inequality
    """
    absolute_frequencies = np.array(absolute_frequencies_list)
    total = 0
    for i, xi in enumerate(absolute_frequencies[:-1], 1):
        total += np.sum(np.abs(xi - absolute_frequencies[i:]))
    gc = total / (len(absolute_frequencies)**2 * np.mean(absolute_frequencies))
    return gc

def stats_pipeline(attribute_elements):
    """Pipeline to calculate all pertinant diversity statistics

    Parameters
    ----------
    attribtute_elements: dict
        Unique elements of an image attribute as keys and instances of each as values

    Returns
    -------
    s: int
        Richness
    h: float
        Shannon Index
    nme: float
        Normalized Median Evenness
    j: float
        Pielou's Evenness
    gc: float
        Gini coefficient
    """
    # Richness
    s = len(attribute_elements.keys())

    # Shannon Index
    rel_frequencies, abs_frequencies = category_frequencies(attribute_elements=attribute_elements)
    h, pi_list = h_index(p=rel_frequencies)

    # Calculate Normalized Median Evenness
    nme = norm_median_evenness(pi_list)

    # Calculate Pielou's evenness
    j = pielou(h=h, s=s)

    # Calculate Gini coefficient
    gc = gini_coef(abs_frequencies)

    return s, h, nme, j, gc
    

In [9]:
# Define stat collection pipeline
def collect_study_stats(metadata_file_path, results_list, na_cols=["pixel_size_x", "pixel_size_y"]):
    """Collecting statistics within a single file"""

    # Read parquet into pandas df
    metadata_df = pd.read_parquet(metadata_file_path)

    # Extract metadata from file name and dataframe
    metadata_pq = metadata_file_path.name
    study_name = metadata_pq.split('.')[0]
    attribute_names = metadata_df.columns.to_list()

    # Remove irrelevant attributes
    for attribute in na_cols:
        attribute_names.remove(attribute)

    # Collect statistics for each attribute
    for attribute in attribute_names:
        unique_entries = metadata_df[attribute].unique()
        attribute_elements = dict()
        for element in unique_entries:
            attribute_elements[element] = len(metadata_df[metadata_df[attribute] == element])

        s, h, nme, j, gc = stats_pipeline(attribute_elements=attribute_elements)

        # Append stats to attribute_results
        results_list.append([study_name,
                                attribute, 
                                s,
                                h, 
                                nme, 
                                j,
                                gc])

    return stat_results_df

In [95]:
 # Collect metadata

 # Define study metadata directory
studies_metadata_dir = pathlib.Path(
    '../data/metadata')

# Collect metadata file paths
metadata_files = list(walk(studies_metadata_dir))

all_results_list = list()
 # Initialize Pool object
start = time.time()
available_cores = len(os.sched_getaffinity(0))
pool = multiprocessing.Pool(processes=available_cores)
print(
    f"\nNow processing {len(metadata_files)} studies with {available_cores} cpu cores.\n"
)

# Build the iterative object for pool.starmap()
metadata_file_paths = list(zip(metadata_files, all_results_list))


for metadata_path in metadata_file_paths:
    collect_study_stats(metadata_path, all_results_list)

stat_results_df = pd.DataFrame(data=all_results_list, columns=['Study_Name', 'Attribute', 'S', 'H', 'NME', 'J', 'GC'])
print(stat_results_df)

# Make directories
stats_dir = pathlib.Path("../data/statistics")
pathlib.Path.mkdir(stats_dir, exist_ok=True)

# Save individual stats as parquet file
output_file = pathlib.Path(
        screen_dir, f"individual_studies_diversity.parquet.gzip"
    )
stat_results_df.to_parquet(output_file, compression="gzip")



Now processing 3 studies with 16 cpu cores.



/tmp/ipykernel_7064/2030695575.py:47: RuntimeWarning: invalid value encountered in double_scalars
  nme = ndimage.median(temp_list) / temp_list.max()
/tmp/ipykernel_7064/2030695575.py:38: RuntimeWarning: invalid value encountered in double_scalars
  return h / ln(s)
/tmp/ipykernel_7064/2030695575.py:47: RuntimeWarning: invalid value encountered in double_scalars
  nme = ndimage.median(temp_list) / temp_list.max()
/tmp/ipykernel_7064/2030695575.py:38: RuntimeWarning: invalid value encountered in double_scalars
  return h / ln(s)
/tmp/ipykernel_7064/2030695575.py:47: RuntimeWarning: invalid value encountered in double_scalars
  nme = ndimage.median(temp_list) / temp_list.max()
/tmp/ipykernel_7064/2030695575.py:38: RuntimeWarning: invalid value encountered in double_scalars
  return h / ln(s)


                                 Study_Name             Attribute      S  \
0     idr0080-way-perturbation_screenA_2701             screen_id      1   
1     idr0080-way-perturbation_screenA_2701            study_name      1   
2     idr0080-way-perturbation_screenA_2701              plate_id     18   
3     idr0080-way-perturbation_screenA_2701            plate_name     18   
4     idr0080-way-perturbation_screenA_2701               well_id   6912   
5     idr0080-way-perturbation_screenA_2701        imaging_method      1   
6     idr0080-way-perturbation_screenA_2701                sample      1   
7     idr0080-way-perturbation_screenA_2701              organism      1   
8     idr0080-way-perturbation_screenA_2701         organism_part      1   
9     idr0080-way-perturbation_screenA_2701             cell_line      3   
10    idr0080-way-perturbation_screenA_2701                strain      1   
11    idr0080-way-perturbation_screenA_2701       gene_identifier     53   
12    idr008

In [21]:
def collect_databank_stats(metadata_dir, na_cols=["pixel_size_x", "pixel_size_y"]):
    """Statistics pipeline for computation accross a databank
    """
    metadata_directory = pathlib.Path("../data/metadata")

    # Open and concatinate study metadata dataframes from .parquet files
    databank_metadata = pd.concat([pd.read_parquet(study_metadata_file) for study_metadata_file in walk(metadata_directory)])
    
    # Get image_attribute names
    attribute_names = databank_metadata.columns.to_list()

    # Remove irrelevant attributes
    for attribute in na_cols:
        attribute_names.remove(attribute)

    results_list = list()
    # Collect statistics for each attribute
    for attribute in attribute_names:
        unique_entries = databank_metadata[attribute].unique()
        attribute_elements = dict()
        for element in unique_entries:
            attribute_elements[element] = len(databank_metadata[databank_metadata[attribute] == element])

        s, h, nme_result, j, gc = stats_pipeline(attribute_elements=attribute_elements)

        # Append stats to attribute_results
        results_list.append([attribute, 
                                s,
                                h, 
                                nme_result, 
                                j,
                                gc])
    
    stat_results_df = pd.DataFrame(data=results_list, columns=['Attribute', 'S', 'H', 'NME', 'J', 'GC'])

    return stat_results_df


/tmp/ipykernel_7074/1919036818.py:49: RuntimeWarning: invalid value encountered in double_scalars
  nme = ndimage.median(temp_list) / temp_list.max()
/tmp/ipykernel_7074/1919036818.py:40: RuntimeWarning: invalid value encountered in double_scalars
  return h / ln(s)


               Attribute      S          H       NME         J        GC
0              screen_id      3   0.860492  0.812077  0.783253  0.363842
1             study_name      3   0.860492  0.812077  0.783253  0.363842
2               plate_id    334   5.494304  0.330586  0.945478  0.371400
3             plate_name    334   5.494304  0.330586  0.945478  0.371400
4                well_id  28320  10.251324  1.000000  1.000000  0.000000
5         imaging_method      2   0.646920  0.880417  0.933309  0.150847
6                 sample      1  -0.000000       NaN       NaN  0.000000
7               organism      3   0.800038  0.820953  0.728226  0.379190
8          organism_part      1  -0.000000       NaN       NaN  0.000000
9              cell_line      5   1.128628  0.730221  0.701256  0.465085
10                strain   3010   4.762870  0.001838  0.594638  0.728306
11       gene_identifier   3058   5.548350  0.001902  0.691339  0.715078
12  phenotype_identifier     13   0.568956  0.05190

In [ ]:
# Collect databank stats
databank_stats = collect_databank_stats(metadata_dir=studies_metadata_dir)
print(databank_stats)